Import Libraries

In [72]:
import os
import msoffcrypto
import pandas as pd
import numpy as np
from datetime import datetime as dt
from matplotlib import pyplot as plt
from scipy.interpolate import CubicSpline
from sklearn.linear_model import LinearRegression
import pmdarima as pm
from pmdarima import model_selection
import math
import pytz
import datetime

In [82]:
xls = pd.ExcelFile("RawData2.xlsx")
Factor_data = pd.read_excel(xls,'Factor')
Gold_data = pd.read_excel(xls,'GoldBar_Sale')

In [83]:
factor_col = Factor_data.columns[1:]

In [84]:
Gold_Factor_data = \
                pd.merge(
                    Gold_data,
                    Factor_data,
                    left_on = 'Report Date',
                    right_on = 'Report Date',
                    how = 'left'
                )

In [85]:
Gold_Factor_data.replace([np.inf, -np.inf],np.nan,inplace=True)
Gold_Factor_data = Gold_Factor_data.dropna(axis='columns')

In [102]:
Gold_Factor_data['LOGIT'] = np.log(Gold_Factor_data['GoldBar_Sale'].values/Gold_Factor_data['GoldBar_Sale'].shift(1))
Gold_Factor_data['LOGIT'].replace([np.inf, -np.inf, np.nan],0,inplace=True)

C:\Users\temp5\AppData\Local\Temp\ipykernel_5104\4255634414.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  Gold_Factor_data['LOGIT'].replace([np.inf, -np.inf, np.nan],0,inplace=True)


In [103]:
Gold_Factor_data['LOGIT']

0      0.000000
1      0.032462
2      0.000000
3      0.061925
4      0.016037
         ...   
151    0.006270
152    0.020109
153    0.046381
154   -0.017700
155    0.001487
Name: LOGIT, Length: 156, dtype: float64

เลือกตัวแปรที่สนใจ และเกี่ยวข้องกับทองคำ

In [109]:
import statsmodels.api as sm
from statsmodels.compat import lzip
from statsmodels.tsa.stattools import adfuller
main_Factor_list = factor_col
# Factor_list = Gold_Factor_data.drop(columns = ['GoldBar_Sale'] ).columns
Factor_list = Gold_Factor_data.drop(columns = ['Report Date','GoldBar_Sale','LOGIT']).columns
All_result_table = pd.DataFrame()
All_result_table_best = pd.DataFrame()

for ix , temp_factor_group in enumerate(main_Factor_list,1):
    Factor_group = [i for i in Factor_list if temp_factor_group in i]
    r2_list = []
    print(ix , " " , temp_factor_group , " " , Factor_group)
    # print(Factor_group)
    temp_result_table = pd.DataFrame()
    for temp_factor in Factor_group:
        X_train = np.array(Gold_Factor_data[temp_factor].values , dtype=float)
        y_train = np.array(Gold_Factor_data['LOGIT'].values , dtype=float)
        
        X_train_1 = X_train.reshape(-1,1)
        y_train_1 = y_train.reshape(-1,1)

        # test stationary test 10%
        adf = adfuller(X_train)[1]
        adf_result = 'PASS' if adfuller(X_train)[1] <= 0.4 else 'FAIL'

        #Regression
        linear_model = LinearRegression()
        linear_model.fit(X_train_1 , y_train_1)

        X_train = sm.add_constant(X_train)
        reg = sm.OLS(y_train , X_train).fit()
        sign = '+' if linear_model.coef_ > 0 else '-'

        r2_list.append((temp_factor,reg.rsquared,sign,adf,adf_result))
        print(r2_list)

    temp_result_table['Factor'] = np.array(r2_list)[:,0]
    temp_result_table['r2'] = np.array(r2_list)[:,1]
    temp_result_table['sign'] = np.array(r2_list)[:,2]
    temp_result_table['adf'] = np.array(r2_list)[:,3]
    temp_result_table['adf_result'] = np.array(r2_list)[:,4]

    temp_result_table_sort = temp_result_table.sort_values(['adf_result','r2'], ascending = [False,False]).reset_index(drop=True)

    if ix == 1:
        All_result_table = temp_result_table
        # All_result_table_best = temp_result_table_sort.head(1)
    else:
        All_result_table = pd.concat([All_result_table,temp_result_table], ignore_index = True)
        # All_result_table_best = pd.concat([All_result_table,temp_result_table_sort.head(1)], ignore_index = True)
All_result_table

1   (Inflation)   ['(Inflation)']
[('(Inflation)', 2.0074462920627312e-05, '-', 0.384208513196113, 'PASS')]
2   แก๊สโซฮอล์ 95   ['แก๊สโซฮอล์ 95']
[('แก๊สโซฮอล์ 95', 0.0038992120111775996, '-', 0.4433370559392254, 'FAIL')]
3   แก๊สโซฮอล์ 91   ['แก๊สโซฮอล์ 91']
[('แก๊สโซฮอล์ 91', 0.003239831477415911, '-', 0.3605919027180712, 'PASS')]
4   แก๊สโซฮอล์ E20   ['แก๊สโซฮอล์ E20']
[('แก๊สโซฮอล์ E20', 0.0022096127180323277, '-', 0.40180837828112953, 'FAIL')]
5   แก๊สโซฮอล์ E85   ['แก๊สโซฮอล์ E85']
[('แก๊สโซฮอล์ E85', 7.81811359181761e-05, '-', 0.7169152371016783, 'FAIL')]
6   ดีเซล   ['ดีเซล']
[('ดีเซล', 0.008121327029269931, '-', 0.23800627119664364, 'PASS')]
7   อัตราแลกเปลี่ยน USD_THB   ['อัตราแลกเปลี่ยน USD_THB']
[('อัตราแลกเปลี่ยน USD_THB', 0.0005678397241831723, '+', 0.2056790383161553, 'PASS')]
8   BOT Policy rate   ['BOT Policy rate']
[('BOT Policy rate', 0.0014587051845749155, '-', 0.08872752547000201, 'PASS')]
9   US Dollar Index   ['US Dollar Index']
[('US Dollar Index', 0.00072510631

,Factor,r2,sign,adf,adf_result
0,(Inflation),2.0074462920627312e-05,-,0.384208513196113,PASS
1,แก๊สโซฮอล์ 95,0.0038992120111775996,-,0.4433370559392254,FAIL
2,แก๊สโซฮอล์ 91,0.003239831477415911,-,0.3605919027180712,PASS
3,แก๊สโซฮอล์ E20,0.0022096127180323277,-,0.40180837828112953,FAIL
4,แก๊สโซฮอล์ E85,7.81811359181761e-05,-,0.7169152371016783,FAIL
5,ดีเซล,0.008121327029269931,-,0.23800627119664364,PASS
6,อัตราแลกเปลี่ยน USD_THB,0.0005678397241831723,+,0.2056790383161553,PASS
7,BOT Policy rate,0.0014587051845749155,-,0.08872752547000201,PASS
8,US Dollar Index,0.0007251063191413554,+,0.5042493483773777,FAIL
9,S&P 500 Index,0.003884991348159339,+,0.9811630268910562,FAIL


In [41]:
Factor_list

,Report Date,GoldBar_Sale
0,2011-01-31,19700
1,2011-02-28,20350
2,2011-03-31,20350
3,2011-04-30,21650
4,2011-05-30,22000
...,...,...
163,2024-05-04,40250
164,2024-05-05,40100
165,2024-05-06,40050
166,2024-05-08,40300


In [112]:
Pass_Factor_list = All_result_table[All_result_table['adf_result'] == 'PASS']
list_Factor.to_excel("ADF_RESULT_PASS.xlsx")

In [113]:
Pass_Factor_list = Pass_Factor_list['Factor']

Overall Model Performance

In [114]:
from itertools import combinations
comb_list = [i for i in combinations(Pass_Factor_list , 3)] + [i for i in combinations(Pass_Factor_list, 2)]

In [120]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from scipy.stats import anderson as AD
from statsmodels.stats.outliers_influence import variance_inflation_factor

all_stats_results = pd.DataFrame()

y_train = np.array(Gold_Factor_data['LOGIT'].values , dtype=float)

for out_index , comb in enumerate(comb_list,1):
    selected_Factor = list(comb)
    X_train = np.array(Gold_Factor_data[selected_Factor].values , dtype = float)

    linear_model = LinearRegression()
    linear_model.fit(X_train , y_train)

    X_train = sm.add_constant(X_train)
    reg = sm.OLS(y_train , X_train).fit()

    X_train = X_train[:,1:]
    y_pred = linear_model.predict(X_train)

    stats_results = pd.DataFrame()
    stats_results[[f'FACTOR_{i+1}' for i in range(len(selected_Factor))]] = np.array([selected_Factor])

    coeff_result = reg.summary2().tables[1].iloc[:,:1].transpose().reset_index(drop=True)
    for i in ['const','x1','x2','x3']:
        if i in coeff_result.columns:
            stats_results[i] = coeff_result[i]
        else:
            stats_results[i] = None
    # P-Value
    stats_results[['intercept_p_value'] + [f'FACTOR_{i+1}_p_value' for i in range(len(selected_Factor))]] = reg.pvalues
    # ADJ Rsq
    r2 = r2_score(y_train,y_pred)
    n , p = X_train.shape
    adj_r2 = 1 - (( 1 - r2 ) * (n - 1) / (n - p - 1))
    stats_results['r2'] = r2
    stats_results['adj_r2'] = adj_r2
    # F stats
    stats_results['Probability of F-Stats'] = reg.summary2().tables[0][3].astype(float)[5]
    # Anderson Darling
    stats_results['AD'] = AD(reg.resid , dist='norm')[0]
    stats_results['AD_criteria'] = [{i:j for i , j in zip(AD( reg.resid , dist='norm')[2],
                                                         AD( reg.resid , dist='norm')[1]
                                                         )}]
    stats_results['AD_criteria_0.05'] = AD( reg.resid , dist='norm')[1][2:3]
    # Breusch-Pagan
    name_stats = ['Lagrange multiplier statistic','p-value','f-value','f p-value']
    test = sm.stats.het_breuschpagan(reg.resid , reg.model.exog )
    stats_results['BP_pvalues'] = lzip(name_stats , test )[1][1]
    # Durbin Watson
    stats_results['Durbin_Watson'] = sm.stats.durbin_watson(reg.resid)
    # VIF 
    VIF_values = [variance_inflation_factor(Gold_Factor_data[selected_Factor].values , i) for i in range(len(selected_Factor))]

    for in_index , i in enumerate(selected_Factor , 0 ):
        stats_results[f'VIF_Factor_{in_index+1}'] = VIF_values[in_index]
    if out_index == 1:
        all_stats_results = stats_results
    else:
        all_stats_results = pd.concat([all_stats_results, stats_results], axis = 0).reset_index(drop=True)
all_stats_results = all_stats_results.sort_values('adj_r2', ascending = False)

C:\Users\temp5\AppData\Local\Temp\ipykernel_5104\3180822270.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stats_results = pd.concat([all_stats_results, stats_results], axis = 0).reset_index(drop=True)
C:\Users\temp5\AppData\Local\Temp\ipykernel_5104\3180822270.py:62: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_stats_results = pd.concat([all_stats_results, stats_results], axis = 0).reset_index(drop=True)
C:\Users\temp5\AppData\Local\Temp\ipykernel_5104\3180822270.py:62: FutureWar

In [139]:
Pass_Stat_list = all_stats_results

In [141]:
#Adj Stats in factor to interesting
Pass_Stat_list = all_stats_results[all_stats_results['r2'] >= 0.00002]
Pass_Stat_list = all_stats_results[all_stats_results['adj_r2'] >= 0.00002]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['Probability of F-Stats'] <= 0.5]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['AD'] < Pass_Stat_list['AD_criteria_0.05']]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['BP_pvalues'] > 0.05]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['Durbin_Watson'] >= 1]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_1'] <= 5]
# Pass_Stat_list = Pass_Stat_list[Pass_Stat_list['VIF_Factor_2'] <= 5]
# Pass_Stat_list = Pass_Stat_list[(Pass_Stat_list['VIF_Factor_3'] <= 5 | (Pass_Stat_list['VIF_Factor_3'].isna()))]

In [142]:
Pass_Stat_list

,FACTOR_1,FACTOR_2,FACTOR_3,const,x1,x2,x3,intercept_p_value,FACTOR_1_p_value,FACTOR_2_p_value,...,adj_r2,Probability of F-Stats,AD,AD_criteria,AD_criteria_0.05,BP_pvalues,Durbin_Watson,VIF_Factor_1,VIF_Factor_2,VIF_Factor_3
11,(Inflation),ดีเซล,NaN,0.045238,0.004486,-0.001691,NaN,0.122939,0.360297,0.149498,...,0.000655,0.352,1.834473,"{15.0: 0.562, 10.0: 0.64, 5.0: 0.768, 2.5: 0.8...",0.768,0.336632,2.194423,3.641017,3.641017,NaN
